<a href="https://colab.research.google.com/github/ShrishRajGupta/ContriHUB-22/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install and Import Dependencies

In [1]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

## 2. Instantiate Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

## 3. Encode and Calculate Sentiment

In [4]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [5]:
result = model(tokens)

In [6]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [7]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

## 4. Collect Reviews

In [8]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]
reviews

['The food was delicious. The ricotta pancakes were light and tasty.  The cream brûlée French toast was a fruit lovers delight. There was fresh strawberries, blueberries, passion fruit and custard. No syrup required. And crispy kale as a side ordered with eggs. Worth the walk over the Pyrmont bridge.',
 "Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 "We came for brunch and they ran out of seven separate menu items. We tried ordering multiple times and 

In [9]:
import numpy as np
import pandas as pd
df = pd.DataFrame(np.array(reviews), columns=['review'])
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
df
df.to_csv('reviews.csv', index=False)

In [10]:
df

,review,sentiment
0,The food was delicious. The ricotta pancakes w...,4
1,Very cute coffee shop and restaurant. They hav...,4
2,We came for brunch and they ran out of seven s...,2
3,Six of us met here for breakfast before our wa...,4
4,Great place with delicious food and friendly s...,5
5,Great food amazing coffee and tea. Short walk ...,5
6,"Great service, lovely location, and really ama...",5
7,Ricotta hot cakes! These were so yummy. I ate ...,5
8,We came for brunch twice in our week-long visi...,4
9,It was ok. Had coffee with my friends. I'm new...,3


## 5 Saving trained model

In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
model_save_name = 'sentiment.pt'
path = F"/content/gdrive/MyDrive/{model_save_name}"
torch.save(model.state_dict(), path)

In [17]:
model_save_name = 'sentiment.pt'
path = F"/content/gdrive/MyDrive/{model_save_name}"
model.load_state_dict(torch.load(path))

<All keys matched successfully>